In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [9]:
# Read the Excel file
df = pd.read_excel("/content/Data Dictionary.xlsx")

# Save it as a CSV file
df.to_csv("/content/Data_Dictionary.csv", index=False)  # index=False to avoid adding an extra index column

print("Conversion complete! CSV saved as 'Data_Dictionary.csv'")

Conversion complete! CSV saved as 'Data_Dictionary.csv'


In [11]:
filename = '/content/Data_Dictionary.csv'
df = pd.read_csv(filename)


**Inspecting the Dataset**

In [14]:
df.info()
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   DataType     32 non-null     object
 1   Instrument   31 non-null     object
 2   Field        31 non-null     object
 3   Description  31 non-null     object
 4   Type         31 non-null     object
 5   Labels       11 non-null     object
dtypes: object(6)
memory usage: 1.9+ KB


,DataType,Instrument,Field,Description,Type,Labels
0,QUANTITATIVE METADATA,NaN,NaN,NaN,NaN,NaN
1,NaN,Identifier,participant_id,Participant ID,str,NaN
2,Quantitative,Edinburgh Handedness Questionnaire,EHQ_EHQ_Total,Laterality Index (Score),float,-100 = 10th left\n−28 ≤ LI < 48 = middle\n100 ...
3,Quantitative,Ishihara Color Vision Test,ColorVision_CV_Score,Color vision test score,int,NaN
4,Quantitative,Alabama Parenting Questionnaire - Parent Report,APQ_P_APQ_P_CP,Corporal Punishment Score,int,NaN
5,Quantitative,Alabama Parenting Questionnaire - Parent Report,APQ_P_APQ_P_ID,Inconsistent Discipline Score,int,NaN
6,Quantitative,Alabama Parenting Questionnaire - Parent Report,APQ_P_APQ_P_INV,Involvement Score,int,NaN
7,Quantitative,Alabama Parenting Questionnaire - Parent Report,APQ_P_APQ_P_OPD,Other Discipline Practices Score (Not factored...,int,NaN
8,Quantitative,Alabama Parenting Questionnaire - Parent Report,APQ_P_APQ_P_PM,Poor Monitoring/Supervision Score,int,NaN
9,Quantitative,Alabama Parenting Questionnaire - Parent Report,APQ_P_APQ_P_PP,Positive Parenting Score,int,NaN


In [15]:
# Check missing values
print(df.isnull().sum())

DataType        5
Instrument      6
Field           6
Description     6
Type            6
Labels         26
dtype: int64


In [18]:
# Identify numeric columns
numeric_cols = df.select_dtypes(include=['number']).columns

# Identify categorical columns
categorical_cols = df.select_dtypes(include=['object']).columns




In [19]:
# Fill missing values for numeric columns using median
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

# Fill missing values for categorical columns using mode
for col in categorical_cols:
    df[col] = df[col].fillna(df[col].mode()[0])  # Fill with most frequent value


In [20]:
print(df.isnull().sum())


DataType       0
Instrument     0
Field          0
Description    0
Type           0
Labels         0
dtype: int64


**Group by DataType**

In [23]:
print(df.columns)


Index(['DataType', 'Instrument', 'Field', 'Description', 'Type', 'Labels'], dtype='object')


In [26]:
df.to_csv("/content/cleaned_dataset.csv", index=False)
print("Dataset cleaned and saved as 'cleaned_dataset.csv'")


Dataset cleaned and saved as 'cleaned_dataset.csv'


In [27]:
df.head(10)

,DataType,Instrument,Field,Description,Type,Labels
0,QUANTITATIVE METADATA,Strength and Difficulties Questionnaire,participant_id,Participant ID,int,"0=Homemaker, stay at home parent.\n5=Day labor..."
1,Quantitative,Identifier,participant_id,Participant ID,str,"0=Homemaker, stay at home parent.\n5=Day labor..."
2,Quantitative,Edinburgh Handedness Questionnaire,EHQ_EHQ_Total,Laterality Index (Score),float,-100 = 10th left\n−28 ≤ LI < 48 = middle\n100 ...
3,Quantitative,Ishihara Color Vision Test,ColorVision_CV_Score,Color vision test score,int,"0=Homemaker, stay at home parent.\n5=Day labor..."
4,Quantitative,Alabama Parenting Questionnaire - Parent Report,APQ_P_APQ_P_CP,Corporal Punishment Score,int,"0=Homemaker, stay at home parent.\n5=Day labor..."
5,Quantitative,Alabama Parenting Questionnaire - Parent Report,APQ_P_APQ_P_ID,Inconsistent Discipline Score,int,"0=Homemaker, stay at home parent.\n5=Day labor..."
6,Quantitative,Alabama Parenting Questionnaire - Parent Report,APQ_P_APQ_P_INV,Involvement Score,int,"0=Homemaker, stay at home parent.\n5=Day labor..."
7,Quantitative,Alabama Parenting Questionnaire - Parent Report,APQ_P_APQ_P_OPD,Other Discipline Practices Score (Not factored...,int,"0=Homemaker, stay at home parent.\n5=Day labor..."
8,Quantitative,Alabama Parenting Questionnaire - Parent Report,APQ_P_APQ_P_PM,Poor Monitoring/Supervision Score,int,"0=Homemaker, stay at home parent.\n5=Day labor..."
9,Quantitative,Alabama Parenting Questionnaire - Parent Report,APQ_P_APQ_P_PP,Positive Parenting Score,int,"0=Homemaker, stay at home parent.\n5=Day labor..."


In [34]:
# Clean up DataType column: remove spaces and standardize case
df["DataType"] = df["DataType"].str.strip().str.lower()

# Replace inconsistent labels
df["DataType"] = df["DataType"].replace({
    "categorical ": "categorical",
    "quantitative metadata": "quantitative",
    "categorical metadata": "categorical",
    "targets": "target"
})

# Check unique values again
print(df["DataType"].unique())



['quantitative' 'categorical' 'target']


In [35]:
# Get quantitative columns
quantitative_df = df[df["DataType"] == "quantitative"]

# Get categorical columns
categorical_df = df[df["DataType"] == "categorical"]

# Get target columns
target_df = df[df["DataType"] == "target"]

print("Quantitative Fields:", quantitative_df["Field"].tolist())
print("Categorical Fields:", categorical_df["Field"].tolist())
print("Target Fields:", target_df["Field"].tolist())

Quantitative Fields: ['participant_id', 'participant_id', 'EHQ_EHQ_Total', 'ColorVision_CV_Score', 'APQ_P_APQ_P_CP', 'APQ_P_APQ_P_ID', 'APQ_P_APQ_P_INV', 'APQ_P_APQ_P_OPD', 'APQ_P_APQ_P_PM', 'APQ_P_APQ_P_PP', 'SDQ_SDQ_Conduct_Problems', 'SDQ_SDQ_Difficulties_Total', 'SDQ_SDQ_Emotional_Problems', 'SDQ_SDQ_Externalizing', 'SDQ_SDQ_Generating_Impact', 'SDQ_SDQ_Hyperactivity', 'SDQ_SDQ_Internalizing', 'SDQ_SDQ_Peer_Problems', 'SDQ_SDQ_Prosocial', 'MRI_Track,Age_at_Scan', 'participant_id', 'participant_id', 'participant_id', 'participant_id']
Categorical Fields: ['participant_id', 'Basic_Demos_Enroll_Year', 'Basic_Demos_Study_Site', 'PreInt_Demos_Fam_Child_Ethnicity', 'PreInt_Demos_Fam_Child_Race', 'MRI_Track_Scan_Location', 'Barratt_Barratt_P1_Edu', 'Barratt_Barratt_P1_Occ', 'Barratt_Barratt_P2_Edu', 'Barratt_Barratt_P2_Occ']
Target Fields: ['participant_id', 'ADHD_Outcome', 'Sex_F']
